# Google Drive Scraper
### Douyin Video Data Extraction/Translation

This Google Colab notebook scrapes a Google Drive folder containing Douyin videos, extracts metadata (title, date, duration), translates titles to English, and generates a spreadsheet with hyperlinks to the videos.

Here's a breakdown in bullet points:
* **Authenticates**: Connects to Google Drive and Google Translate API using service account credentials.
* **Traverses Files**: Walks through a specified Google Drive folder containing subfolders for each company and their Douyin videos.
* **Extracts Metadata**: Extracts the video title and upload date from the filename.
* **Translates**: Translates the video titles (Chinese to English using Google Translate API).
* **Calculates Duration**: Calculates the video duration using the moviepy library.
* **Generates Links**: Retrieves Google Drive web view links for each video.
* **Creates Spreadsheet**: Organizes all extracted data into a Pandas DataFrame and exports it to an Excel file with clickable hyperlinks to the videos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install google-api-python-client pandas ffmpeg-python

In [ ]:
# Hook up to G Drive
from google.oauth2 import service_account
from googleapiclient.discovery import build

creds = service_account.Credentials.from_service_account_file('credentials.json')
drive_service = build('drive', 'v3', credentials=creds)


In [ ]:
# Folder traversal for ids
def find_folder_id(service, folder_name, parent_folder_id):
    try:
        # Query to search within the specified parent folder
        query = (
            f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder'"
            f" and '{parent_folder_id}' in parents"
        )

        # print(f"Searching for folder: {folder_name} with query: {query}")
        results = service.files().list(q=query, supportsAllDrives=True, includeItemsFromAllDrives=True, fields="files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print(f"No folder found with name: {folder_name} in parent ID: {parent_folder_id}")
            return None

        # Take the first matching folder
        # print(f"Found folder with ID: {items[0]['id']}")
        return items[0]['id']

    except Exception as error:
        print(f"An error occurred: {error}")
        return None

In [ ]:
# Set helper functions
# Function to format seconds as hh:mm:ss
def format_duration(seconds):
    try:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours:02}:{minutes:02}:{secs:02}"
    except TypeError:
        return "00:00:00"  # Default for missing or invalid durations

# Function to translate text to English
def translate_to_english(text):
    try:
        result = translate_service.translations().list(
            q=text,
            target='en'
        ).execute()
        return result['translations'][0]['translatedText']
    except Exception as e:
        print(f"Translation error for '{text}': {e}")
        return text  # Return the original text if translation fails

In [ ]:
# Constants
import os
import pandas as pd
from moviepy.editor import VideoFileClip
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Authenticate with Google Translate API
translate_creds = service_account.Credentials.from_service_account_file('credentials.json')
translate_service = build('translate', 'v2', credentials=translate_creds)

# Specify the root folder in Google Drive
oo_dir_TEST = "local"
oo_dir_TEST_folderId = "1jtuO8NuGO3ZCTFIi73MGb3JN3rtkIH6_"

oo_dir_1 = "Downloaded Douyin videos"
oo_dir_1_folderId = "11ECto7u2RGXz3tRRPzcaUqv4ovwl7FUt"

oo_dir_2 = "Downloaded Douyin videos for aquaculture companies"
oo_dir_2_folderId = "1DJacV51PrVI1MZLX7saYAWIG0a8rIQSr"

oo_dir_3 = "Downloaded Douyin videos for aquaculture companies/主页作品"
oo_dir_3_folderId = "14NXizuyC6W1vFGhvt3w2RXYMI7cSuRY-"

oo_dir_4 = "Downloaded Douyin videos for aquaculture companies/主页作品 (Nov 24)"
oo_dir_4_folderId = "1VPzJAPzENtF9dOHs03aO2NXw6ysqPB_e"

ROOT = "/content/drive/MyDrive/outlaw-ocean/"

In [ ]:
# Build file mapping to webview link
def fetch_files_in_folder(folder_id):
    """Retrieve all file names and IDs in a folder."""
    files = []
    page_token = None
    while True:
        # print(f"Fetching files in folder with ID: {folder_id}")
        response = drive_service.files().list(
            q=f"'{folder_id}' in parents",
            fields="nextPageToken, files(id, name)",
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,  # Includes files from shared drives
            pageToken=page_token
        ).execute()
        files.extend(response.get('files', []))
        page_token = response.get('nextPageToken')
        if not page_token:
            break
    # print(f"Fetched {len(files)} files in total, see: {files}")
    return files

def get_file_info(file_id):
    """Retrieve file metadata using its unique ID."""
    try:
        file = drive_service.files().get(
            fileId=file_id,
            fields="id, name, webViewLink",
            supportsAllDrives=True,
        ).execute()
        return file
    except Exception as e:
        print(f"Error retrieving file info for ID {file_id}: {e}")
        return None

def build_name_to_id_map(folder_id):
    """Build a mapping of file names to IDs for a folder."""
    files = fetch_files_in_folder(folder_id)
    return {file['name']: file['id'] for file in files}

In [ ]:
# Real meat & potatoes
# Initialize data list
data = []

# Starting parameters
isTranslate = True
isDuration = True
isExcel = True
root_folder = ROOT + oo_dir_4

# Map local dirpath to Drive folder IDs (manual or through API)
dirpath_to_drive_id = {
    ROOT + oo_dir_TEST : oo_dir_TEST_folderId,
    ROOT + oo_dir_1 : oo_dir_1_folderId,
    ROOT + oo_dir_2 : oo_dir_2_folderId,
    ROOT + oo_dir_3 : oo_dir_3_folderId,
    ROOT + oo_dir_4 : oo_dir_4_folderId
}

# Traverse files
MAX_FILES = 6600;
num_files = 0;
num_dirs = 0
parent_folder = dirpath_to_drive_id.get(root_folder)
for dirpath, dirnames, filenames in os.walk(root_folder):
    print(f"Processing directory: {dirpath}")
    if num_files > MAX_FILES:
        break

    company = os.path.basename(dirpath)  # Folder name as company

    # Get the folder ID for this directory
    folder_id = find_folder_id(drive_service, company, parent_folder)

    # Build file -> webview link mapping
    if not folder_id:
        print(f"Folder not found for company: {company}")
        folder_id = parent_folder
        continue

    num_dirs += 1
    name_to_id_map = build_name_to_id_map(folder_id)

    # Convert company name to English
    translated_company = translate_to_english(company) if isTranslate else company

    for file in filenames:
        if file.endswith(('.mp4', '.mov', '.avi')):  # Add more extensions if needed
            num_files += 1
            if num_files > MAX_FILES:
                break
            filepath = os.path.join(dirpath, file)

            # Extract title: Match between the '-' and the '.mp4'
            title = file[file.find('-') + 1:file.rfind('.mp4')].strip()

            date = file.split('-')[0]  # Extract date before dash

            # Translate the title to English
            translated_title = translate_to_english(title) if isTranslate else title

            # Get hyperlink for the file
            file_id = name_to_id_map.get(file)
            if not file_id:
                print(f"File ID not found for file: {file}")
                 # Try looser key examination
                for key in name_to_id_map.keys():
                    if key[:15] == file[:15]:  # Compare the first 15 characters
                        file_id = name_to_id_map[key]
                        print(f"Looser match found: {file_id} for key: {key}")
                        break
                if (file_id == None):
                    print(f"No matching file ID found for file: {file}")
                    hyperlink = ""
            else :
                file_info = get_file_info(file_id)
                hyperlink = file_info['webViewLink'] if file_info else ""

            # Calculate video duration and format as hh:mm:ss
            try:
                if isDuration:
                    clip = VideoFileClip(filepath, audio=False)
                    duration_seconds = clip.duration
                    duration_formatted = format_duration(duration_seconds)
                    clip.close()
                else:
                    duration_formatted = "00:00:00"
            except Exception as e:
                duration_formatted = "00:00:00"  # Default for errors

            # Append the extracted data
            data.append({
                "Company": company,
                "Translated Company" : translated_company,
                "Date": date,
                "Hyperlink": hyperlink if hyperlink else "",
                "Title": title,
                "Translated Title": translated_title,
                "Duration": duration_formatted
            })
    print(f"Completed {num_dirs} directories")

# Create a DataFrame
df = pd.DataFrame(data)

# Convert to hyperlinks
if (isExcel):
  df['Hyperlink'] = df['Hyperlink'].apply(lambda url: f'=HYPERLINK("{url}", "link")' if url else "")
  # df['Hyperlink'] = df['Hyperlink'].apply(lambda url: url if url else "")
  df.to_excel('/content/video_index_with_hyperlinks_主页作品 (Nov 24).xlsx', index=False)
else :
  # Save to a CSV file
  df.to_csv('/content/video_index_with_translation.csv', index=False)

print("Complete!")

